In [ ]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint
import os
from bs4 import BeautifulSoup
import requests
import json
import jsonschema
from jsonschema import validate
import re
from datetime import datetime

### Git Client connection to Azure DevOps

The code below uses connects to your Organization and uses a Personal Access Token (PAT) to authenticate to Azure DevOps  

You will need to provide:
1. the url to your organization 
2. a PAT against the organization that is listed in the URL


In [ ]:
# Get a client (the "core" client provides access to projects, teams, etc)
def get_clients_projects():

    # Fill in with your personal access token and org URL
    personal_access_token = os.getenv("gitpat")
    organization_url = os.getenv("org_url")

    # Create a connection to the org
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)    
    
    core_client = connection.clients.get_core_client()
    git_client = connection.clients.get_git_client()

    # Get the first page of projects
    get_projects_response = core_client.get_projects()
    return core_client, git_client, get_projects_response

### Simple JSON validation

This is not 100% accurrate but is sufficient for most use cases in testing whether a string is in fact a valid JSON string.

In [ ]:
def json_validator(data):
    try:
        json.loads(data)
        return True
    except ValueError as error:
        print("invalid json: %s" % error)
        return False

### Reading all JSON files in the pipeline folder

The following will read all files in the pipeline folder for every repository in an organization. Further, it will modify only the specified branch in any repository that has a matching name. The ADF Activity type that is is specifically looking for is the "copy" activity. If you need to search on more then a for loop should be created to iterate through all the activities.

In [ ]:
def get_all_keys(d):
    for key, value in d.items():
        yield key
        if isinstance(value, dict):
            yield from get_all_keys(value)

In [ ]:
def look_in_string(search_string, search_value, end_value):
    i=0
    search_pairs = []
    while i < len(search_string):
        start = i
        end = len(search_string)
        try:
            start = search_string.index(search_value,start,end)
            carry = search_string.index(end_value,start+1,end)
            found_value = search_string[start:carry]
            search_pairs.append(found_value)
            i += start

        except:
            print('==================>>>>>>>>>i guess i threw an error<<<<<<<==================')
            i = len(search_string)

    return search_pairs


In [ ]:
def update_repository(file_name):
    core_client, git_client, get_projects_response = get_clients_projects()

    from base64 import b64encode
    from base64 import decode
    import requests

    from requests.auth import HTTPBasicAuth

    branch_to_find = 'test1'

    # whole number
    retryIntervalInSeconds = 80

    # whole number
    retry = 33

    # string value formatted as Days, Hours, Minutes, Seconds {0.0:00:00}
    timeout = "0.0:4:00"

    git_commit = dict()

    index = 0
    while get_projects_response is not None:
        for project in get_projects_response.value:
            repos = git_client.get_repositories(project.id)
            for repo in repos:
                try:
                    branches = git_client.get_branches(repository_id=repo.id)
                    for branch in branches:
                        if branch.name != branch_to_find:
                            #print(f'These are not the droids you are looking for: {branch.name}')
                            continue

                        commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)
                        tree = git_client.get_tree(repository_id=repo.id, sha1=commit.tree_id)

                        for entry in tree.tree_entries:
                            if entry.relative_path == "pipeline":
                                cc = git_client.get_tree(repository_id=repo.id, sha1=entry.object_id)


                                

                                for c in cc.tree_entries:
                                    #if file_name != c.relative_path:
                                    #    continue
                                    

                                

                                    if c.git_object_type == 'blob':
                                        try:

                                            blob = git_client.get_blob(repository_id=repo.id, sha1=c.object_id)

                                            r = requests.get(blob.url, auth=HTTPBasicAuth('',os.getenv("gitpat")))

                                            content = json.dumps(json.loads(r.content))

                                            what_i_found = look_in_string(content,"\"timeout\"",",")
                                            for found in what_i_found:
                                                content = content.replace(found,f'"timeout": \"{timeout}\"')

                                            what_i_found = look_in_string(content,"\"retryIntervalInSeconds\"",",")
                                            for found in what_i_found:
                                                content = content.replace(found,f'"retryIntervalInSeconds": {retryIntervalInSeconds}')

                                            what_i_found = look_in_string(content,"\"retry\"",",")
                                            for found in what_i_found:
                                                content = content.replace(found,f'"retry": {retry}')

                                        except Exception as ex:
                                            print(f'failed to read json file {ex}')

                                        try:
                                            content = json.loads(content)
                                        except Exception as mm:
                                            print(f'error on converting to dict: {mm}')


                                        my_json = json.dumps(content,indent=4)

                                        file_path = f"/{entry.relative_path +'/'+ c.relative_path}"

                                        stage_dict = dict({file_path:my_json})
                                        git_commit.update(stage_dict)


                                        #push = {"refUpdates":[{"name":"refs/heads/" + branch.name,"oldObjectId": commit.commit_id }],"commits":[{"comment":"Updating readme via python and Azure DevOps SDK: " + datetime.now().strftime("%Y %B, %A %w, %H hours %M minutes"),"changes":[{"changeType":"edit","item":{"path":"/" + entry.relative_path + "/" + c.relative_path},"newContent":{"content": my_json ,"contentType":"rawText"}}]}]}

                                        #git_client.create_push(
                                        #    repository_id=repo.id, push=push
                                        #)


                except Exception as e:
                    print(e)
            index += 1

            #project.
        if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
            # Get the next page of projects
            get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

        else:
            # All projects have been retrieved
            get_projects_response = None

    return git_commit

In [ ]:
core_client, git_client, get_projects_response = get_clients_projects()

from base64 import b64encode
from base64 import decode
import requests

from requests.auth import HTTPBasicAuth

branch_to_find = 'test1'

# whole number
retryIntervalInSeconds = 81

# whole number
retry = 10

# string value formatted as Days, Hours, Minutes, Seconds {0.0:00:00}
timeout = "0.0:21:00"

files_in_pipeline=[]
index = 0
while get_projects_response is not None:
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            try:
                branches = git_client.get_branches(repository_id=repo.id)
                for branch in branches:
                    if branch.name != branch_to_find:
                        continue

                    commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)
                    tree = git_client.get_tree(repository_id=repo.id, sha1=commit.tree_id)

                    for entry in tree.tree_entries:
                        if entry.relative_path == "pipeline":
                            cc = git_client.get_tree(repository_id=repo.id, sha1=entry.object_id)
                            for c in cc.tree_entries:
                                files_in_pipeline.append(c.relative_path)

            except Exception as e:
                print(e)
        index += 1

        #project.
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None


In [ ]:
for file_name in files_in_pipeline:
    print(file_name)
    update_repository(file_name=file_name)

In [ ]:
output = update_repository(file_name='first-pipeline.json')

In [ ]:
for k,v in output.items():
    print(k)

In [ ]:
parameter_1 = "Foo"
parameter_2 = "Bar"
sql_string = 'Hello {parameter_1} this is {parameter_2}; '
formatted_sql = sql_string.format(**locals())
print(sql_string)
print(formatted_sql)

tvars = { 'parameter_1': 'Foo', 'parameter_2': 'Bar' }
formatted_sql = sql_string.format(**tvars)
print(formatted_sql)


In [1]:
#from utils.json_builder import refupdates,commits
from utils.json_builder import Change
import json


mychange = Change(branch_name="test1", commit_id="skdfk293492k2", comment="update adf pipelines")


In [2]:
mychange.add_content(path="pipeline/pipe1.json", content="message one")
mychange.add_content(path="pipeline/pipe2.json", content="message two")
mychange.add_content(path="pipeline/pipe3.json", content="message three")
mychange.add_content(path="pipeline/pipe4.json", content="message four")


In [3]:
fjd = mychange.git_push()
print(fjd)

{"refUpdates": [{"name": "refs/heads/test1", "oldObjectId": "skdfk293492k2"}], "commits": [{"comment": "update adf pipelines{"changeType": "edit", "item": {"path": "/pipeline/pipe1.json"}, "newContent": {"content": "message one", "contentType": "rawtext"}}","changes": [{"changeType": "edit", "item": {"path": "/pipeline/pipe2.json"}, "newContent": {"content": "message two", "contentType": "rawtext"}}","changes": [{"changeType": "edit", "item": {"path": "/pipeline/pipe3.json"}, "newContent": {"content": "message three", "contentType": "rawtext"}}","changes": [{"changeType": "edit", "item": {"path": "/pipeline/pipe4.json"}, "newContent": {"content": "message four", "contentType": "rawtext"}}]}]}


In [ ]:
f'
{
  "refUpdates": [
    {
      "name": "refs/heads/{branch.name}",
      "oldObjectId": "{commit.commit_id}"
    }
  ],
  "commits": [
    {
      "comment": "Updating files: { datetime.now().strftime("%Y %B, %A %w, %H hours %M minutes")}",
      "changes": [
        {
          "changeType": "edit",
          "item": {
            "path": "/" {entry.relative_path + "/" + c.relative_path}"
          },
          "newContent": {
            "content": "{my_json}",
            "contentType": "rawtext"
          }
        },
        {
          "changeType": "add",
          "item": {
            "path": "/tasks/content/newtasks.md"
          },
          "newContent": {
            "content": "# New Tasks\n\nTBD",
            "contentType": "rawtext"
          }
        },
        {
          "changeType": "add",
          "item": {
            "path": "/tasks/content/inactivetasks.md"
          },
          "newContent": {
            "content": "# Inactive Tasks\n\nTBD",
            "contentType": "rawtext"
          }
        }
      ]
    }
  ]
}
'

In [ ]:
'
 {"refUpdates":[{"name":"refs/heads/" + branch.name,"oldObjectId": commit.commit_id }],"commits":[{"comment":"Updating readme via python and Azure DevOps SDK: " + datetime.now().strftime("%Y %B, %A %w, %H hours %M minutes"),"changes":[{"changeType":"edit","item":{"path":"/" + entry.relative_path + "/" + c.relative_path},"newContent":{"content": my_json ,"contentType":"rawText"}}]}]}
'